In [91]:
# importing dependencies 
import pandas as pd
import numpy as np
from difflib import get_close_matches
import re
from config import api_key
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [92]:
# initalizing OpenCageGeocode
geocoder = OpenCageGeocode(api_key)

In [93]:
replace_list = [
    [
        "Between ",
        ""
    ],
    [
        "Subdistrict",
        "Sub District"
    ],
    [
        "Island Group",
        ""
    ],
    [
        "Provine",
        "Province"
    ],
    [
        " Atoll",
        ""
    ],
    [
        "miles from ",
        ""
    ],
    [
        "miles north of ",
        ""
    ],
    [
        "miles off ",
        ""
    ],
    [
        "miles southeast of ",
        ""
    ],
    [
        "miles west of ",
        ""
    ],
    [
        "miles SE of ",
        ""
    ],
    [
        "miles east of ",
        ""
    ],
    [
        "nm from ",
        ""
    ],
    [
        "nm southeast of ",
        ""
    ],
    [
        "nm st of ",
        ""
    ],
    [
        "to mileswest of the ",
        ""
    ],
    [
        "to miles ea off ",
        ""
    ],
    [
        " fresh water ",
        ""
    ],
    [
        " Pacific Ocean",
        ""
    ],
    [
        "miles offshore",
        ""
    ],
    [
        "Great Exuma Island",
        "Great Exuma"
    ],
    [
        "Guerrrero",
        "Guerrero"
    ],
    [
        "Guerro",
        "Guerrero"
    ],
    [
        "Great Exuma Island",
        "Great Exuma"
    ],
    [
        "Alinglaplap",
        "Alifu Alifu"
    ],
    [
        "Golfo di Genova in the ",
        ""
    ],
    [
        "Golfo de ",
        ""
    ],
    [
        "Inhambe",
        "Inhambane"
    ],
    [
        "Galica",
        "Galicia"
    ],
    [
        " between Port Edward and Port St Johns",
        ""
    ],
    [
        "Leyte Island",
        "Leyte"
    ],
    [
        "Liguaria",
        "Liguria"
    ],
    [
        "Ligurian Sea",
        "Liguria"
    ],
    [
        "miles southwest of O ahu ",
        ""
    ],
    [
        "the north coast",
        ""
    ],
    [
        "to Taboguilla Island Pacific Ocean",
        "Taboguilla Island"
    ],
    [
        "Corfu Island",
        "Corfu"
    ],
    [
        "Corfu ",
        "Corfu"
    ],
    [
        "Cyclades archipelago",
        "Cyclades"
    ],
    [
        "Enroute from Suez to Aden Yemen ",
        "Aden"
    ],
    [
        "nm west of the ",
        ""
    ],
    [
        "miles south of ",
        ""
    ],
    [
        "km off the coast of Iran km from mouth of ",
        ""
    ],
    [
        "San Blas coast",
        "San Blas"
    ],
    [
        "San Blas Islands",
        "San Blas"
    ],
    [
        "Vita Levu",
        "Viti Levu"
    ],
    [
        "Viti Levu Island",
        "Viti Levu"
    ],
    [
        "Viti Levu group",
        "Viti Levu"
    ],
    [
        "Vitu Levu",
        "Viti Levu"
    ],
    [
        " and U S A",
        ""
    ],
    [
        "East of the ",
        ""
    ],
    [
        "Off the coast of ",
        ""
    ],
    [
        "Off the western coast of peninsular ",
        ""
    ],
    [
        "Rombion",
        "Romblon"
    ],
    [
        "Tavenui",
        "Taveuni"
    ],
    [
        "Tuamotos",
        "Tuamotus"
    ],
    [
        "  province",
        ""
    ],
    [
        " arm of the North Sea",
        ""
    ],
    [
        " near Karitane north of Dunedin",
        ""
    ],
    [
        "Shat Al Arab River",
        "Shatt al Arab River"
    ],
    [
        "Shatt el Arab River",
        "Shatt al Arab River"
    ],
    [
        " les Bains",
        ""
    ],
    [
        "Saint Beno t",
        "Saint Benoit"
    ],
    [
        "Argyllshire",
        "Argyll"
    ],
    [
        "Makora Ulawa Province",
        "Makira Ulawa Province"
    ],
    [
        "Makira Uluwa Province",
        "Makira Ulawa Province"
    ],
    [
        "Somewhere between Philadelphia and ",
        ""
    ],
    [
        "Near the ",
        ""
    ],
    [
        "Near ",
        ""
    ],
    [
        "Off coast of ",
        ""
    ],
    [
        "Off ",
        ""
    ],
    [
        "the Coromandel Peninsula North Island",
        "Coromandel"
    ],
    [
        " in the Gulf of Leyte",
        ""
        
    ],
    
]

In [94]:
# def in_nested_list(my_list, item):
#     """
#     Determines if an item is in my_list, even if nested in a lower-level list.
#     """
#     return any(in_nested_list(sublist, item) for sublist in my_list if isinstance(sublist, list))

def clean_data(x):
    x = x.lstrip()
    x = re.sub("[^a-zA-Z]", ' ', x)
    x = re.sub(' +', ' ', x)
    x = x.lstrip()
    
    for sub_list in replace_list:
        x = x.replace(sub_list[0], sub_list[1])
    x = x.lstrip()
    x = x.rstrip()
    
    x = re.sub('^(to |To )', '', x)
    x = re.sub('^(Southern )', '', x)
    x = re.sub('^(East )', '', x)
    x = re.sub('^(Western )', '', x)
    x = re.sub('(Count)$', 'County', x)
    return x
    

In [95]:
# Cleaning national shark data
sharkAttacks = pd.read_csv('data/attacks.csv', encoding='cp1252').drop(columns=['Unnamed: 22', 'Unnamed: 23'])

sharkAttacks = sharkAttacks.dropna(subset=['Area'])
sharkAttacks['CleanArea'] = sharkAttacks.Area.apply(str)
sharkAttacks.CleanArea = sharkAttacks.CleanArea.apply(clean_data)

df1 = sharkAttacks.CleanArea + ", " + sharkAttacks.Country

df1 = df1.astype(str)

cleaned_areas_list = df1.unique().tolist()
cleaned_areas_list.sort()
# cleaned_areas_list


# len(thing)
# for t in thing:
#     print(f'{t} coordinates')


In [96]:
# Reading in txt file 
file_data = open("loc_data.txt", "r")
data = file_data.read()
# convert string from txt file to dictionary 
if data:
    location_data = eval(data)
else:
    # creating empty dict
    location_data = {}
    
# iterating through list cleaned_areas_list and checking if they are in location_data dict
# adding items that are not in dict to dict 
for item in cleaned_areas_list:
    query = f'{item}'
    if item not in location_data:
        location_data[item] = geocoder.geocode(query)
        
# writing dict to txt file
file = open("loc_data.txt","w")
file.write( str(location_data) )
file.close()

In [141]:
file_data = open("loc_data.txt", "r")
data = file_data.read()

data = eval(data)
response_dict = {}
for key, value in data.items():
    if value:
        response_dict[key] = value[0].get("geometry")

# loc_df = pd.DataFrame(response_dict.items())
loc_df.reset_index()

loc_df.columns = ['CombinedArea', 'Geography']

sharkAttacks["CombinedArea"] = sharkAttacks.CleanArea + ", " + sharkAttacks.Country

# df.columns = ['a', 'b']

# sharkAttacks

# merged_df = pd.merge(sharkAttacks, loc_df, left_on='CleanArea', right_on='Area')
merged_df = sharkAttacks.merge(loc_df, left_on=['CombinedArea'], right_on=['CombinedArea'], how='left')
merged_df[['CombinedArea','Geography']]

# sharkAttacks
loc_df
# loc_df


,CombinedArea,Geography
0,", DOMINICAN REPUBLIC","{'lat': 19.0974031, 'lng': -70.3028026}"
1,", PANAMA","{'lat': 8.559559, 'lng': -81.1308434}"
2,", TASMAN SEA","{'lat': -40.0, 'lng': 160.0}"
3,"Abaco Islands, BAHAMAS","{'lat': 26.3918096, 'lng': -77.1370018}"
4,"Abau Sub District Central Province, NEW GUINEA","{'lat': -5.0, 'lng': 140.0}"
...,...,...
761,"aint Denis, REUNION","{'lat': -21.10084, 'lng': 55.60344}"
762,"coast, USA","{'lat': 37.858337, 'lng': -122.2447394}"
763,"d tang Sal, REUNION","{'lat': -21.10084, 'lng': 55.60344}"
764,"land, ATLANTIC OCEAN","{'lat': -45.0, 'lng': -15.0}"


In [129]:
# np.where(pd.isnull(merged_df))
data_merged = merged_df[merged_df["Geography"].isna()][["Geography", "Country", "Location"]]
data_merged

,Geography,Country,Location
103,NaN,"ST HELENA, British overseas territory",English Bay
142,NaN,"ST HELENA, British overseas territory",English Bay
854,NaN,TURKS & CAICOS,Mudjin Harbor
860,NaN,TURKS & CAICOS,French Cay
1732,NaN,TURKS & CAICOS,French Cay
2833,NaN,NaN,NaN
3245,NaN,NaN,Between St. Kitts & Nevis
3827,NaN,NaN,NaN
4034,NaN,NaN,NaN
4066,NaN,NaN,Geyser Bank


In [130]:
merged_df["Geography"][103] = "{'lat': -15.9655, 'lng': -5.7115}"
merged_df["Geography"][103]

/Users/autumperconti/opt/anaconda3/envs/Project3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


"{'lat': -15.9655, 'lng': -5.7115}"

In [131]:
merged_df["Geography"][142] = "{'lat': -15.9655, 'lng': -5.7115}"
merged_df["Geography"][142]

/Users/autumperconti/opt/anaconda3/envs/Project3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


"{'lat': -15.9655, 'lng': -5.7115}"

In [132]:
merged_df["Geography"][860] = "{'lat': 21.500, 'lng': -72.217}"
merged_df["Geography"][860]

/Users/autumperconti/opt/anaconda3/envs/Project3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


"{'lat': 21.500, 'lng': -72.217}"

In [133]:
merged_df["Geography"][1732] = "{'lat': 21.500, 'lng': -72.217}"
merged_df["Geography"][1732]

/Users/autumperconti/opt/anaconda3/envs/Project3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


"{'lat': 21.500, 'lng': -72.217}"

In [134]:
merged_df["Geography"][854] = "{'lat': 21.788192, 'lng': -71.734542}"
merged_df["Geography"][854]

/Users/autumperconti/opt/anaconda3/envs/Project3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


"{'lat': 21.788192, 'lng': -71.734542}"

In [135]:
merged_df["Geography"][3245] = "{'lat': 17.363747, 'lng': -62.754593}"
merged_df["Geography"][3245]

/Users/autumperconti/opt/anaconda3/envs/Project3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


"{'lat': 17.363747, 'lng': -62.754593}"

In [136]:
merged_df["Geography"][4131] = "{'lat': 40.946888, 'lng': 54.495243}"
merged_df["Geography"][4131]

/Users/autumperconti/opt/anaconda3/envs/Project3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


"{'lat': 40.946888, 'lng': 54.495243}"

In [137]:
merged_df["Geography"][4410] = "{'lat': 13, 'lng': 122}"
merged_df["Geography"][4410]

/Users/autumperconti/opt/anaconda3/envs/Project3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


"{'lat': 13, 'lng': 122}"

In [138]:
merged_df["Geography"][4744] = "{'lat': -38.73, 'lng': 77.522222}"
merged_df["Geography"][4744]

/Users/autumperconti/opt/anaconda3/envs/Project3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


"{'lat': -38.73, 'lng': 77.522222}"

In [139]:
merged_df["Geography"][4281] = "{'lat': 15.748, 'lng': -81.75343}"
merged_df["Geography"][4281]

/Users/autumperconti/opt/anaconda3/envs/Project3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


"{'lat': 15.748, 'lng': -81.75343}"

In [140]:
data_merged

,Geography,Country,Location
103,NaN,"ST HELENA, British overseas territory",English Bay
142,NaN,"ST HELENA, British overseas territory",English Bay
854,NaN,TURKS & CAICOS,Mudjin Harbor
860,NaN,TURKS & CAICOS,French Cay
1732,NaN,TURKS & CAICOS,French Cay
2833,NaN,NaN,NaN
3245,NaN,NaN,Between St. Kitts & Nevis
3827,NaN,NaN,NaN
4034,NaN,NaN,NaN
4066,NaN,NaN,Geyser Bank


In [126]:
data_merged.to_sql('Geo', con=engine, if_exists='replace',
           index_label='id')
engine.execute("SELECT * FROM Geo").fetchall()

NameError: name 'engine' is not defined